##Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import os
from tqdm import tqdm
import pickle
import numpy as np
from sklearn.cluster import KMeans
import shutil
from PIL import Image
import random
import matplotlib.pyplot as plt

Mounted at /content/drive


## (failed) Converting .npz to a pandas Series



In [ ]:
ALL_AERIAL_IMAGES_DIR = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/aerial_embeddings_all"

image_files = []

# Use tqdm to monitor progress during file iteration
for filename in tqdm(os.listdir(ALL_AERIAL_IMAGES_DIR), desc="Scanning files"):
    if os.path.isfile(os.path.join(ALL_AERIAL_IMAGES_DIR, filename)):
        image_files.append(filename)

image_series = pd.Series(image_files)

Scanning files: 100%|██████████| 32183/32183 [01:58<00:00, 272.19it/s] 


In [ ]:
ALL_STREETVIEW_IMAGES_DIR = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/streetview_embeddings_all"

image_files = []

# Use tqdm to monitor progress during file iteration
for filename in tqdm(os.listdir(ALL_STREETVIEW_IMAGES_DIR), desc="Scanning files"):
    if os.path.isfile(os.path.join(ALL_STREETVIEW_IMAGES_DIR, filename)):
        image_files.append(filename)

image2_series = pd.Series(image_files)

Scanning files: 100%|██████████| 32269/32269 [00:08<00:00, 3981.61it/s]


**There 3709 aerial embeddings in julia's random streetview folder**

In [ ]:
streetview_dir = "drive/MyDrive/streetview_embeddings"

image_files = []

# Use tqdm to monitor progress during file iteration
for filename in tqdm(os.listdir(streetview_dir), desc="Scanning files"):
    if os.path.isfile(os.path.join(streetview_dir, filename)):
        image_files.append(filename)

image_series = pd.Series(image_files)

Scanning files: 100%|██████████| 3709/3709 [00:00<00:00, 6847.37it/s]


In [ ]:
julia_aerial = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/aerial_embeddings_julia"
athena_aerial = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/aerial_embeddings_athena"
eric_aerial = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/aerial_embeddings_eric"

image_files = []

# Use tqdm to monitor progress during file iteration
for filename in tqdm(os.listdir(julia_aerial), desc="Scanning files"):
    if os.path.isfile(os.path.join(julia_aerial, filename)):
        image_files.append(filename)

# Use tqdm to monitor progress during file iteration
for filename in tqdm(os.listdir(athena_aerial), desc="Scanning files"):
    if os.path.isfile(os.path.join(athena_aerial, filename)):
        image_files.append(filename)

# Use tqdm to monitor progress during file iteration
for filename in tqdm(os.listdir(eric_aerial), desc="Scanning files"):
    if os.path.isfile(os.path.join(eric_aerial, filename)):
        image_files.append(filename)

image_series = pd.Series(image_files)

Scanning files: 100%|██████████| 14608/14608 [00:10<00:00, 1375.70it/s]


In [ ]:
image_series.shape

(33014,)

In [ ]:
duplicates = image_series[image_series.duplicated()]
duplicates.shape

(745,)

In [ ]:
image_series_unique = image_series.drop_duplicates()
image_series_unique.shape

(32269,)

In [ ]:
image_series_unique.head()

,0
0,8_Lord_Street_Coventry_CV5_8DA.npz
1,68_Fifth_Avenue_Westminster_W10_4DP.npz
2,118_Barns_Road_Oxford_OX4_3RF.npz
3,1_Willow_Grove_Coventry_CV4_9EP.npz
4,38_Shirley_Road_Coventry_CV2_2EN.npz


In [ ]:
output_directory = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/aerial_embeddings.pkl"

with open(output_directory,"wb") as f:
  pickle.dump(image_series_unique,f)

print("series has been saved")

series has been saved


In [ ]:
# Load the Series from the pickle file
with open(output_directory, "rb") as f:
    loaded_series = pickle.load(f)

print("Loaded Series:")
print(loaded_series)

Loaded Series:
0                       8_Lord_Street_Coventry_CV5_8DA.npz
1                  68_Fifth_Avenue_Westminster_W10_4DP.npz
2                        118_Barns_Road_Oxford_OX4_3RF.npz
3                      1_Willow_Grove_Coventry_CV4_9EP.npz
4                     38_Shirley_Road_Coventry_CV2_2EN.npz
                               ...                        
33009                  84_Alfall_Road_Coventry_CV2_3GG.npz
33010              47_Manfield_Avenue_Coventry_CV2_2QF.npz
33011      Flat_9_27_Bolton_Street_Westminster_W1J_8BW.npz
33012     1_Westbury_Mews_Briton_Road_Coventry_CV2_4RT.npz
33013    Flat_5C_4_Craven_Hill_Gardens_Westminster_W2_3...
Length: 32269, dtype: object


##Extract from .npz into .npy

downloading

In [6]:
ALL_IMAGES_DIR = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/aerial_embeddings_all"
extracted_folder = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/aerial_extracted_npy"

# Create folder for extracted .npy files
if not os.path.exists(extracted_folder):
    os.makedirs(extracted_folder)

all_files = [file_name for file_name in os.listdir(ALL_IMAGES_DIR)];

# Extract and save .npy arrays
for file_name in tqdm(all_files[983:]):
    if file_name.endswith(".npz"):
        data = np.load(os.path.join(ALL_IMAGES_DIR, file_name))
        np.save(os.path.join(extracted_folder, file_name.replace(".npz", ".npy")), data['arr_0'])

print("Extraction complete! All .npy files are saved in", extracted_folder)

  2%|▏         | 514/31714 [03:47<3:50:31,  2.26it/s]


KeyboardInterrupt: 

## K-Means Clustering ⭐

make clusters

In [ ]:
ALL_STREETVIEW_IMAGES_DIR = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/streetview_extracted_npy"
output_folder = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/filtered_streetview_embeddings_julia"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

embeddings = []
file_paths = []

for file_name in tqdm(os.listdir(ALL_STREETVIEW_IMAGES_DIR), desc = "Processing files"):
  data = np.load(os.path.join(ALL_STREETVIEW_IMAGES_DIR, file_name))
  embeddings.append(data)  # Assuming embeddings are in arr_0.npy
  file_paths.append(file_name)  # Keep track of filenames

embeddings = np.vstack(embeddings)

Processing files: 100%|██████████| 32269/32269 [02:32<00:00, 211.37it/s]


In [ ]:
# Perform K-Means clustering
n_clusters = 20  #adjust as necessary
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(embeddings)

visualize some clusters

In [ ]:
ALL_STREETVIEW_IMAGES_DIR = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/streetview"

cluster_to_images = {i: [] for i in range(n_clusters)}

for label, file_name in zip(labels, file_paths):
    cluster_to_images[label].append(file_name)

for cluster, images in cluster_to_images.items():
    print(f"Cluster {cluster}:")
    sample_images = random.sample(images, min(20, len(images)))

    # grid for display
    num_columns = 5
    num_rows = (len(sample_images) + num_columns - 1) // num_columns  # calculate rows

    plt.figure(figsize=(num_columns * 2, num_rows * 2))  #make size smaller

    for i, img_file in enumerate(sample_images):
        image_name = img_file.replace(".npy", ".jpg")
        img = Image.open(os.path.join(ALL_STREETVIEW_IMAGES_DIR, image_name))

        plt.subplot(num_rows, num_columns, i + 1)  #subplot
        plt.imshow(img)
        plt.axis("off")

    plt.tight_layout()  #avoid overlaps
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

remove unwanted clusters and export

In [ ]:
EXTRACTED_STREETVIEW_IMAGES = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/streetview_extracted_npy"
output_folder = "drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/filtered_streetview_embeddings_julia"

unwanted_clusters = [5,7,9,15,17]  #replace with indexes of clusters to remove

with tqdm(total=len(file_paths), desc = "Filtering and copying npy files") as pbar:
  for label, file_name in zip(labels, file_paths):
      if label not in unwanted_clusters:
          shutil.copy(os.path.join(EXTRACTED_STREETVIEW_IMAGES, file_name), output_folder)
      pbar.update(1)

print("Filtered dataset exported to", output_folder)

Filtering and copying npy files: 100%|██████████| 32269/32269 [06:22<00:00, 84.40it/s]

Filtered dataset exported to drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/filtered_streetview_embeddings_julia


In [ ]:
num_files = len(os.listdir(output_folder))
print(f"Number of files in '{output_folder}':", num_files)

Number of files in 'drive/MyDrive/semester 4/csci 1470: final project/datasetimages/train/filtered_streetview_embeddings_julia': 27979
